# Higgs Example 

## Load modules

In [ ]:
import os
import ROOT
from ROOT import TMVA
os.environ["KERAS_BACKEND"] = "tensorflow"

## I/O

In [ ]:
ROOT.TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize();

outputFile = ROOT.TFile.Open("Higgs_ClassificationOutput.root", "RECREATE")

factory = ROOT.TMVA.Factory("TMVA_Higgs_Classification", outputFile,
                            "!V:ROC:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" )

## Load Data and Features

In [ ]:
loader = ROOT.TMVA.DataLoader("dataset")

loader.AddVariable("m_jj")
loader.AddVariable("m_jjj")
loader.AddVariable("m_lv")
loader.AddVariable("m_jlv")
loader.AddVariable("m_bb")
loader.AddVariable("m_wbb")
loader.AddVariable("m_wwbb")

## Define Inputs

In [ ]:
inputFileName = "data/Higgs_data.root"

inputFile = ROOT.TFile.Open( inputFileName )

signalTree     = inputFile.Get("sig_tree")
backgroundTree = inputFile.Get("bkg_tree")

signalWeight     = 1.0
backgroundWeight = 1.0
   
loader.AddSignalTree    ( signalTree,     signalWeight     )
loader.AddBackgroundTree( backgroundTree, backgroundWeight )

mycuts = ROOT.TCut("")  
mycutb = ROOT.TCut("") 

loader.PrepareTrainingAndTestTree( mycuts, mycutb,
                                   "nTrain_Signal=7000:nTrain_Background=7000:SplitMode=Random:"
                                   "NormMode=NumEvents:!V" )

## Book Algorithms (Boosted Decisions Trees, Random Forest)

In [ ]:
## TMVA Boosted Decision Trees
factory.BookMethod(loader,ROOT.TMVA.Types.kBDT, "BDT",
                   "!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:"
                   "BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )

## Scikit-learn Gradient-Boosted Trees
factory.BookMethod(loader, ROOT.TMVA.Types.kPyGTB, "PyGTB","H:!V:VarTransform=G:NEstimators=400:LearningRate=0.1:"
                                                  "MaxDepth=3")
## Scikit-learn Random Forest
factory.BookMethod(loader, ROOT.TMVA.Types.kPyRandomForest, "PyRandomForest","!V:VarTransform=G:NEstimators=400:"
                           "Criterion=gini:MaxFeatures=auto:MaxDepth=6:MinSamplesLeaf=3:MinWeightFractionLeaf=0:"
                            "Bootstrap=kTRUE" )
## Scikit-learn AdaBoosted Trees      
factory.BookMethod(loader, ROOT.TMVA.Types.kPyAdaBoost, "PyAdaBoost","!V:VarTransform=G:NEstimators=400" )


## Book Algorithms (Neural Networks)

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape

In [ ]:
# Define model
model = Sequential()
model.add(Dense(64, kernel_initializer='glorot_normal', activation='tanh', input_dim=7))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='tanh'))
model.add(Dense(64, kernel_initializer='glorot_normal', activation='tanh'))
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['categorical_accuracy',])
model.save('model_dense.h5')
model.summary()

In [ ]:
factory.BookMethod(loader, ROOT.TMVA.Types.kPyKeras, 'Keras_Dense',
                   'H:!V:VarTransform=G:FilenameModel=./model_dense.h5:'+\
                   'NumEpochs=30:BatchSize=32:TriesEarlyStopping=10')

## Train Methods

In [ ]:
factory.TrainAllMethods();

## Test  Methods

In [ ]:
factory.TestAllMethods();   

## Evaluate Methods

In [ ]:
factory.EvaluateAllMethods();

## Plot ROC Curve

In [ ]:
%jsroot on
c1 = factory.GetROCCurve(loader);
c1.Draw();